# Calculating agricultural receipts from pumping information

In [ ]:
import pandas as pd
import geopandas as gp
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

### we need to navigate to the base run to get well IDs and to the `econ` directory for some information needed to link it all up

In [ ]:
base_run_path = Path("../pycap_runs/pycap_base")
econ_path = Path('../econ')

### first get the well IDs from the pycap yml configuration file

In [ ]:
with open(base_run_path / "LPR_Redux.yml", 'r') as ifp:
        indat = yaml.safe_load(ifp)

In [ ]:
# now we need to strip off just the numbers into a list
wells = [int(i.split('_')[1]) for i in list(indat.keys()) if i.startswith('well')]
wells[:5]

### we need to cross reference these IDs to align with the other datasets below. The IDs in the Yml file are hicap IDs - we need generic site numbers. This happens because numbers get scrambled to alias information and make it harder to link up individual peoples' property with water use and other information.

In [ ]:
well_source_xref = pd.read_csv(econ_path / "source_hcw_xref.csv", index_col=1).dropna()
well_source_xref.index = [int(i) if not np.isnan(i) else 0 for i in well_source_xref.index]

In [ ]:
# but this is a "crosswalk" between well ID (the index) and site_no (the column)
well_source_xref

In [ ]:
src_wells = [well_source_xref.loc[w, 'site_no'] 
             for w in wells
             if w in well_source_xref.index ]

In [ ]:
len(src_wells), len(wells)

### So the `src_wells` list above is the `site_no` values corresponding with the well_ids we have in our model. Note that, of the 327 wells in our `pycap` model, only 231 have corresponding information on parcels and crop rotations. That's due to the crop rotation and parcel information being a bit dated. We will just have to work with what we have, though.

### There are a number of crop rotations identified in the area. Let's check out a shapefile

In [ ]:
crop_rotations = gp.read_file('../econ/parcel_rot.shp').dropna()

In [ ]:
# there are 5 rotations defined
crop_rotations.rotation.unique()

In [ ]:
crop_rotations.explore(column='rotation')

### note that this covers a larger area - it's from a bigger project. But, now we have parcels identified with their crop rotations also identified. #nice So, we just need to sort out which wells supply which parcels. If only there were some kind of a lookup table........

In [ ]:
source2parcel = pd.read_csv('../econ/source_parcel_xref.csv')
source2parcel

### 👆👆👆 That's a lookup table linking `site_no` with parcel_id.....and parcel_id is in the shapefile above along with a defined crop rotation. So, to calculate receipts, we will need to know:
### 0. which parcels are supplied by each well
### 1. how many acres are irrigated in that parcel
### 2. what is grown there (e.g. the crop rotation)
### 3. what is the amount of money in receipts generated per acre by each crop


In [ ]:
wells_w_acres = pd.DataFrame(index=src_wells)

In [ ]:
wells_w_acres = wells_w_acres.merge(source2parcel, left_index=True, right_on='site_no')
wells_w_acres = wells_w_acres.merge(crop_rotations[['parcel_id', 'rotation', 'ag_acres']],
                                    left_on='parcel_id', right_on='parcel_id')

In [ ]:
wells_w_acres.rotation.unique()

In [ ]:
wells_w_acres


### now we need to calculate the per-acre dollar value for each rotation

#### source for rotation percentages:
https://widnr.widen.net/view/pdf/z8j5lsfp00/DG_CSLSAppendixF_2021.pdf?t.download=true
https://p.widencdn.net/8ghipa/Wiscland_2_User_Guide_September_2016

#### source for per acre costs
http://nass.usda.gov/Statistics_by_State/Wisconsin/Publications/Annual_Statistical_Bulletin/2024AgStats_WI.pdf

#### The following table from the sources listed above, indicates the percent of crops making up each rotation

In [ ]:
rotations = pd.read_excel(econ_path / 'CropValuation.xlsx', 
                          sheet_name='Rotations',
                          index_col=1).drop(columns="Crop")
rotations

### Then we also have the dollars of receipts per acre per crop

In [ ]:
per_crop_values = pd.read_excel(econ_path / 'CropValuation.xlsx', 
                          sheet_name='Crop_Values',
                          usecols="F,G", index_col=0).dropna()
per_crop_values

In [ ]:
# now if we bring in all the valuation and merge with the rotations....
rotations = rotations.merge(per_crop_values, left_index=True, right_index=True)

In [ ]:
for cc in rotations:
    if 'dollar' not in cc:
        rotations[cc] *= rotations['dollars_per_acre']

In [ ]:
rotations

In [ ]:
# now sum across crops for each rotation and set non-ag to 0
receipts = rotations[[cc for cc in rotations.columns
                   if 'dollar' not in cc]].sum()
receipts.loc['non-ag'] = 0

In [ ]:
receipts

# now we pull it all together, with `marginal_ag_receipts` defined as the receipts per parcel for each well. Since an individual well (e.g. `site_no`) can supply multiple parcels, we also need to aggregate these in the `groupby` step below.

In [ ]:
wells_w_acres['receipts'] = [receipts.loc[i] for i in wells_w_acres.rotation]

In [ ]:
wells_w_acres['acres_irrigated'] = wells_w_acres.source_to_parcel_pct * wells_w_acres.ag_acres
wells_w_acres['marginal_ag_receipts'] = wells_w_acres['acres_irrigated'] * wells_w_acres.receipts

In [ ]:
wells_w_acres

In [ ]:
receipts_summary = wells_w_acres.groupby('site_no').sum().rename(
    columns={'marginal_ag_receipts':'total_receipts'}
)[['total_receipts','acres_irrigated']]

In [ ]:
receipts_summary

In [ ]:
(receipts_summary.total_receipts/receipts_summary.acres_irrigated).hist(bins=50)
plt.title('Dollars per acre for each well')

In [ ]:
# flip the index back to hi-cap well numbers to align with the yml file
receipts_summary = receipts_summary.merge(well_source_xref,left_index=True,right_on='site_no')

#### now, TL;DR, we have a lookup table with the index being well ID as used by `pycap`, and other columns being total irrigated acres supplied by that well and the total receipts. Important assumption here is that we will later assume that the amount of receipts would be achieved by the well pumping at its assumed rate. When we calculate receipts later, we will simply linearly increasde or decrease receipts by the same fraction that well pumping is increased or reduced. There is nuance beyond that but for this exercise, we will leave it here.

In [ ]:
receipts_summary.to_csv(econ_path / 'total_receipts.csv')